In [2]:
import numpy as np

 Fix dimension (m has to be an odd integer):

In [104]:
m= 3

Define base fields and matrix spaces over these fields:

In [105]:
F2m = GF(2^m)
F22 = GF(2^2)
F22m = GF(2^(2*m))
F2 = GF(2)
M = MatrixSpace(F2, m,m)
Mm = MatrixSpace(F2m, m,m)

Define normal basis $\alpha$ and its generator $\alpha_0$ using GAP:

In [106]:
# normal basis

alpha = [F2m(element) for element in list(gap(F2m).NormalBase())]

alpha

[z3 + 1, z3^2 + 1, z3^2 + z3 + 1]

In [107]:
# generator

alpha_0 = alpha[0]

alpha_0

z3 + 1

The following function takes as input an element $\mathbb{F}_{2^m}$ and and integer $m$ and outputs the trace of the field extension $\mathbb{F}_{2^m}/\mathbb{F}_{2}$:

In [108]:
def trace(element, dim):
    return sum([element**(2**i) for i in range(0,dim)])

Define permutation matrix $S$:

In [109]:
def S(dim):
    l = []
    for i in range(0,dim):
        for j in range(0,dim):
            if i == mod(j-1,dim):
                l.append(1)
            else:
                l.append(0)
    return(M(np.array(l).reshape((dim,dim))))
    
# test

S(m)

[0 1 0]
[0 0 1]
[1 0 0]

Define matrix $A = \mathrm{tr}(\alpha \alpha')$:

In [110]:
def A(dim,Basis):
    a_prime = list(vector(Basis)*S(dim))
    return Matrix([[trace(element1*element2,dim) for element1 in Basis]for element2 in a_prime])

A = A(m, alpha)

In [111]:
A

[0 0 1]
[1 0 0]
[0 1 0]

Compute inverse $A^{-1}$:

In [112]:
A_inv = A.inverse()
A_inv

[0 1 0]
[0 0 1]
[1 0 0]

Define matrix $D = \sum_{i=0}^{n-1} \alpha_0^{2^i} S^i$:

In [113]:
def D(element, perm, dim):
    return Matrix(sum([ (element**(2**j))*(perm**j) for j in range(0,dim) ]))

D(alpha_0, S(m), m)

[       z3 + 1      z3^2 + 1 z3^2 + z3 + 1]
[z3^2 + z3 + 1        z3 + 1      z3^2 + 1]
[     z3^2 + 1 z3^2 + z3 + 1        z3 + 1]

Define $\omega = (D^2)_{0,0}$:

In [114]:
w = (D(alpha_0, S(m), m)**2)[0,0]

w

z3^2 + 1

Define vector $\mathbf{\omega} = ((D^2)_{0,i})_{i\in [n]}$:

In [115]:
omega = [(D(alpha_0, S(m), m)**2)[0,i] for i in range(0,m) ]
   
omega            

[z3^2 + 1, z3 + 1, z3^2 + z3 + 1]

Now a self-complementary normal basis can be computed via: $\Gamma = \mathbf{\omega}\cdot
A^{-1}$.

In [116]:
SCN = list(vector(omega)*A_inv)

SCN

[z3^2 + z3 + 1, z3^2 + 1, z3 + 1]

Verify that basis is equal to dual basis, i. e. self-complementary:

In [117]:
F2m.dual_basis(SCN) == SCN

True

Extend the basis to the field $\mathbb{F}_{2^{2m}}$. For this take a random element $a \in \mathbb{F}_{2^{2}}$ and compute its square:

In [118]:
extension = []
for element in F22:
    if F22(trace(element,2))==1:
        extension.append(element)
        extension.append(element**2)
        break

extension

[z2, z2 + 1]

Extend SCN basis by multiplying $\{ a, a^2 \}$ and $\alpha$:

In [119]:
SCN_even = flatten([[ F22m(element1*element2) for element1 in extension ]for element2 in SCN])

In [120]:
SCN_even

[z6^3,
 z6^5 + z6^3 + z6 + 1,
 z6^5 + z6^3 + 1,
 z6^5 + z6^4 + z6^3 + z6^2 + z6 + 1,
 z6^5 + z6^3 + z6^2 + z6 + 1,
 z6^4 + z6^3 + z6 + 1]

Check if it is self-complementary:

In [121]:
SCN_even == F22m.dual_basis(SCN_even)

True

Now construct matrix with non-zero determinant. For this choose arbitrary element $\beta \in \mathbb{F}_{2^m}$ such that $\beta \neq 1,0$:

In [122]:
for element in F2m:
    if (element != 0 and element != 1):
        beta = element
        break

        
beta

z3

Define matrix $G = \mathrm{tr}(\alpha_i \beta \alpha_j)$, where $\alpha$ is a SCN basis:

In [123]:
def G(Basis, Beta, dim):
    return matrix([[trace(element1*Beta*element2,dim) for element1 in Basis] for element2 in Basis])
 


Construct $m \times m$ - matrix using the SCN basis of $\mathbb{F}_{2^{m}}/\mathbb{F}_{2}$:

In [124]:
G_odd = G(SCN, beta, m)



Construct $2m \times 2m$ - matrix using the SCN basis of $\mathbb{F}_{2^{2m}}/\mathbb{F}_{2}$:

In [131]:
G_even = G(SCN_even, beta, 2*m)

G_even

[0 0 1 0 0 0]
[0 0 0 1 0 0]
[1 0 1 0 1 0]
[0 1 0 1 0 1]
[0 0 1 0 1 0]
[0 0 0 1 0 1]

Define matrices \begin{align}
       N = \begin{pmatrix}
            G & \mathbb{I}_n \\
            \mathbb{I}_n & G
            \end{pmatrix}
\quad \text{and} \quad 
     N +J = \begin{pmatrix}
            G & 0 \\
            0 & G
            \end{pmatrix}.
    \end{align}

In [132]:
def N(dim, matr):
    return matrix(np.vstack((np.hstack((identity_matrix(dim), np.array(matr))), np.hstack(( np.array(matr), identity_matrix(dim) )))))

Define the $2m \times 2m$-matrix $N$ (odd case):

In [133]:
N(m, G_odd)

[1 0 0 0 1 0]
[0 1 0 1 1 1]
[0 0 1 0 1 1]
[0 1 0 1 0 0]
[1 1 1 0 1 0]
[0 1 1 0 0 1]

Check determinant:

In [134]:
N(m, G_odd).determinant()

1

Define the $4m \times 4m$-matrix $N'$ (even case):

In [135]:
N(2*m, G_even)

[1 0 0 0 0 0 0 0 1 0 0 0]
[0 1 0 0 0 0 0 0 0 1 0 0]
[0 0 1 0 0 0 1 0 1 0 1 0]
[0 0 0 1 0 0 0 1 0 1 0 1]
[0 0 0 0 1 0 0 0 1 0 1 0]
[0 0 0 0 0 1 0 0 0 1 0 1]
[0 0 1 0 0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 1 0 0 0 0]
[1 0 1 0 1 0 0 0 1 0 0 0]
[0 1 0 1 0 1 0 0 0 1 0 0]
[0 0 1 0 1 0 0 0 0 0 1 0]
[0 0 0 1 0 1 0 0 0 0 0 1]

Check determinant:

In [136]:
N(2*m, G_even).determinant()

1

Define matrix $J$:

In [137]:
def J(dim):
    return Matrix(F2, np.hstack((np.vstack((np.zeros((dim, dim)), -identity_matrix(dim))), np.vstack((identity_matrix(dim), np.zeros((dim, dim))))))) 

Compute the $2m \times 2m$-matrix $N+J$ (odd case):

In [138]:
N(m, G_odd)+ J(m)

[1 0 0 1 1 0]
[0 1 0 1 0 1]
[0 0 1 0 1 0]
[1 1 0 1 0 0]
[1 0 1 0 1 0]
[0 1 0 0 0 1]

Check determinant:

In [139]:
(N(m, G_odd)+ J(m)).determinant()

1

Compute the $4m \times 4m$-matrix $N+J$ (even case):

In [140]:
N(2*m, G_even)+ J(2*m)

[1 0 0 0 0 0 1 0 1 0 0 0]
[0 1 0 0 0 0 0 1 0 1 0 0]
[0 0 1 0 0 0 1 0 0 0 1 0]
[0 0 0 1 0 0 0 1 0 0 0 1]
[0 0 0 0 1 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0 0 1 0 0]
[1 0 1 0 0 0 1 0 0 0 0 0]
[0 1 0 1 0 0 0 1 0 0 0 0]
[1 0 0 0 1 0 0 0 1 0 0 0]
[0 1 0 0 0 1 0 0 0 1 0 0]
[0 0 1 0 0 0 0 0 0 0 1 0]
[0 0 0 1 0 0 0 0 0 0 0 1]

Check determinant:

In [141]:
(N(2*m, G_even)+ J(2*m)).determinant()

1

In [149]:
HG = groups.matrix.Heisenberg(n=2, R=GF(2))

In [150]:
HG

Heisenberg group of degree 2 over Finite Field of size 2